#  <span style="color: #4daafc">Legal Case Similarity Detection - Pre-processing</span>

- [Load data](#load-data)
- [Pre-processing & data cleaning](#pre-processing)
- [Save Processed Data](#save-processed-data)

# Environment

In [ ]:
from utils.file_utils import load_file, save_file
from utils.df import df_shape
import re

# Load data

In [2]:
f_path = 'data/all_data_ar_ap.xlsx'
df = load_file(file_name=f_path)

Successfully loaded DataFrame from data/all_data_ar_ap.xlsx


In [3]:
# display first few rows
display(df.head(10))

# print data shape
df_shape(df)

# print columns
print(list(df.columns))

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
0,7618/18,10,"ע""א 7618/18",2018,"ע""א 7618/18","ע""א",ערעור אזרחי,0.0,2019-11-12,381,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
1,6857/18,4,"ע""א 6857/18",2018,"ע""א 6857/18","ע""א",ערעור אזרחי,0.0,2021-03-18,897,...,['שופט'],['מחוזי '],['תל אביב'],1.0,"[['ע""א 6701/18', 'ע""א 6857/18']]",NaN,NaN,NaN,NaN,2018
2,5821/18,2,"ע""פ 5821/18",2018,"ע""פ 5821/18","ע""פ",ערעור פלילי,0.0,2018-11-22,108,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2018
3,2075/18,5,"ע""א 2075/18",2018,"ע""א 2075/18","ע""א",ערעור אזרחי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""א 2033/18', 'ע""א 1751/18', 'ע""א 2075/18']]",NaN,NaN,NaN,NaN,2018
4,4217/18,10,"ע""פ 4217/18",2018,"ע""פ 4217/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
5,2212/18,7,"ע""א 2212/18",2018,"ע""א 2212/18","ע""א",ערעור אזרחי,0.0,2018-12-31,243,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
6,4344/18,2,"ע""פ 4344/18",2018,"ע""פ 4344/18","ע""פ",ערעור פלילי,0.0,2019-08-18,439,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
7,1993/18,4,"ע""פ 1993/18",2018,"ע""פ 1993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""פ 1993/18', 'ע""פ 2017/18']]",NaN,NaN,NaN,NaN,2018
8,4993/18,4,"ע""פ 4993/18",2018,"ע""פ 4993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
9,3128/18,4,"ע""פ 3128/18",2018,"ע""פ 3128/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


Data shape: 46109 rows x 140 columns
['מספר תיק', 'מספר החלטות בתיק', 'שם הליך', 'שנת פתיחת ההליך', 'מספר הליך', 'סוג הליך', 'פירוט סוג הליך', 'תיק חסוי', 'תאריך אירוע אחרון', 'משך התיק בימים', 'תאריך הגשה', 'מתוקן סוג הליך', 'תאריך יצוא התיק', 'תאריך יצוא הקובץ', 'תאריך נרמול הקובץ', 'שם הקובץ', 'קישור לתיק', 'שם גורם שיפוטי', 'תואר גורם שיפוטי', 'מספר גורמים שיפוטיים במותב', 'ID', 'שם השופט', 'שם השופט מנורמל', 'שם משפחה מנורמל', 'שם פרטי מנורמל', 'מגדר', 'לאום', 'דת', 'קבוצה דתית', 'שיוך עדתי', 'ארץ/איזור לידה', 'שנת לידה', 'יישוב מגורים מרכזי בילדות', 'דירוג חברתי כלכלי של  יישוב מגורים מרכזי בילדות', 'בית ספר תיכון', 'סוג החינוך', 'מוסד חינוך משפטי עיקרי', 'שנת סיום התואר הראשון במשפטים', 'סוג השכלה משפטית מתקדמת', 'מוסד חינוך משפטי מתקדם', 'שנת קבלת התואר המתקדם', 'מקום התמחות', 'שנת הסמכה כעורך/ת דין', 'המשרה האחרונה טרם המינוי לשיפוט', 'משרה שיפוטית 1', 'שם בית המשפט במשרה שיפוטית 1', 'שנת מינוי למשרה שיפוטית 1', 'משרה שיפוטית 2', 'שם בית המשפט במשרה שיפוטית 2', 'שנת מינוי למשר

In [4]:
# column renaming
col_translation = {
    'מספר תיק': 'case_number',
    'מספר החלטות בתיק': 'number_of_decisions_in_case',
    'שם הליך': 'procedure_name',
    'שנת פתיחת ההליך': 'year_of_procedure_opening',
    'מספר הליך': 'procedure_number',
    'סוג הליך': 'procedure_type',
    'פירוט סוג הליך': 'procedure_type_details',
    'תיק חסוי': 'confidential_case',
    'תאריך אירוע אחרון': 'date_of_last_event',
    'משך התיק בימים': 'case_duration_in_days',
    'תאריך הגשה': 'submission_date',
    'מתוקן סוג הליך': 'corrected_procedure_type',
    'תאריך יצוא התיק': 'case_export_date',
    'תאריך יצוא הקובץ': 'file_export_date',
    'תאריך נרמול הקובץ': 'file_normalization_date',
    'שם הקובץ': 'file_name',
    'קישור לתיק': 'case_link',
    'שם גורם שיפוטי': 'judicial_entity_name',
    'תואר גורם שיפוטי': 'judicial_entity_title',
    'מספר גורמים שיפוטיים במותב': 'number_of_judicial_entities_in_panel',
    'ID': 'id',
    'שם השופט': 'judge_name',
    'שם השופט מנורמל': 'normalized_judge_name',
    'שם משפחה מנורמל': 'normalized_last_name',
    'שם פרטי מנורמל': 'normalized_first_name',
    'מגדר': 'gender',
    'לאום': 'nationality',
    'דת': 'religion',
    'קבוצה דתית': 'religious_group',
    'שיוך עדתי': 'ethnic_affiliation',
    'ארץ/איזור לידה': 'country_region_of_birth',
    'שנת לידה': 'year_of_birth',
    'יישוב מגורים מרכזי בילדות': 'primary_childhood_residence',
    'דירוג חברתי כלכלי של  יישוב מגורים מרכזי בילדות': 'socioeconomic_rating_of_primary_childhood_residence',
    'בית ספר תיכון': 'high_school',
    'סוג החינוך': 'type_of_education',
    'מוסד חינוך משפטי עיקרי': 'primary_legal_education_institution',
    'שנת סיום התואר הראשון במשפטים': 'year_of_first_law_degree_completion',
    'סוג השכלה משפטית מתקדמת': 'type_of_advanced_legal_education',
    'מוסד חינוך משפטי מתקדם': 'advanced_legal_education_institution',
    'שנת קבלת התואר המתקדם': 'year_of_advanced_degree_completion',
    'מקום התמחות': 'internship_location',
    'שנת הסמכה כעורך/ת דין': 'year_of_lawyer_certification',
    'המשרה האחרונה טרם המינוי לשיפוט': 'last_position_before_judicial_appointment',
    'משרה שיפוטית 1': 'judicial_position_1',
    'שם בית המשפט במשרה שיפוטית 1': 'court_name_in_judicial_position_1',
    'שנת מינוי למשרה שיפוטית 1': 'year_of_appointment_to_judicial_position_1',
    'משרה שיפוטית 2': 'judicial_position_2',
    'שם בית המשפט במשרה שיפוטית 2': 'court_name_in_judicial_position_2',
    'שנת מינוי למשרה שיפוטית 2': 'year_of_appointment_to_judicial_position_2',
    'משרה שיפוטית 3': 'judicial_position_3',
    'שם בית המשפט במשרה שיפוטית 3': 'court_name_in_judicial_position_3',
    'שנת מינוי למשרה שיפוטית 3': 'year_of_appointment_to_judicial_position_3',
    'משרה שיפוטית 4': 'judicial_position_4',
    'שם בית המשפט במשרה שיפוטית 4': 'court_name_in_judicial_position_4',
    'שנת מינוי למשפרה שיפוטית 4': 'year_of_appointment_to_judicial_position_4',
    'משרה שיפוטית 5': 'judicial_position_5',
    'שם בית המשפט במשרה שיפוטית 5': 'court_name_in_judicial_position_5',
    'שנת מינוי למשרה שיפוטית 5': 'year_of_appointment_to_judicial_position_5',
    'משרה שיפוטית 6': 'judicial_position_6',
    'שם בית המשפט במשרה שיפוטית 6': 'court_name_in_judicial_position_6',
    'שנת  מינוי למשרה שיפוטית 6': 'year_of_appointment_to_judicial_position_6',
    'משרה שיפוטית 7': 'judicial_position_7',
    'שם בית המשפט במשרה שיפוטית 7': 'court_name_in_judicial_position_7',
    'שנת מינוי למשרה שיפוטית 7': 'year_of_appointment_to_judicial_position_7',
    'משרה שיפוטית 8': 'judicial_position_8',
    'שם בית המשפט במשרה שיפוטית 8': 'court_name_in_judicial_position_8',
    'שנת מינוי למשרה שיפוטית 8': 'year_of_appointment_to_judicial_position_8',
    'שנת תחילת כהונה ראשונה': 'year_of_first_term_start',
    'שנת סיום כהונה ראשונה': 'year_of_first_term_end',
    'שנת תחילת כהונה שניה': 'year_of_second_term_start',
    'שנת סיום כהונה שניה': 'year_of_second_term_end',
    'כהונה כרשם/מת?': 'tenure_as_registrar',
    'כהונה כשופט/ת מחוזי?': 'tenure_as_district_judge',
    'כהונה כשופט/ת שלום?': 'tenure_as_magistrate_judge',
    'שנת סיום מינוי כשופט/ת': 'year_of_judicial_appointment_end',
    'גיל מינוי לשיפוט': 'age_at_judicial_appointment',
    'ותק במשפטים בעת מינוי ראשון': 'legal_experience_at_first_appointment',
    'גיל בעת הפרישה משיפוט': 'age_at_judicial_retirement',
    'ותק שיפוטי בעת המינוי לביהמש העליון': 'judicial_experience_at_supreme_court_appointment',
    'כל קוד הטמל': 'all_html_code',
    'סוג מסמך': 'document_type',
    'קישור למסמך': 'document_link',
    "צד א'": "party_a",
    "צד ב'": "party_b",
    'סוגי הליכים': 'types_of_procedures',
    'לפני': 'before',
    'גוף המסמך': 'document_body',
    'תאריך החלטה אחרונה או פסק דין': 'date_of_last_decision_or_judgment',
    'מספר מילים בגוף המסמך': 'number_of_words_in_document_body',
    'תאריך': 'date',
    'מספר עמודים': 'number_of_pages',
    'קישור למסמך הטמל': 'html_document_link',
    "בשם צד א'": "on_behalf_of_party_a",
    "בשם צד ב'": "on_behalf_of_party_b",
    'מדור': 'section',
    'סטטוס תיק': 'case_status',
    "צד א' ראשון": "first_party_a",
    "צד ב' ראשון": "first_party_b",
    'אירוע אחרון': 'last_event',
    'מספר צדדים בתיק': 'number_of_parties_in_case',
    'מספר תיקי דלמטה בתיק': 'number_of_sub_cases_in_case',
    'מספר דיונים בתיק': 'number_of_hearings_in_case',
    'מספר אירועים בתיק': 'number_of_events_in_case',
    'מספר אישורי מסירה בתיק': 'number_of_delivery_confirmations_in_case',
    'מספר בקשות בתיק': 'number_of_requests_in_case',
    'סוג צד': 'party_type',
    'מספר צד': 'party_number',
    'שם צד': 'party_name',
    'באי כוח צד': 'party_representatives',
    'מזהה צד': 'party_identifier',
    'תאריך דיון': 'hearing_date',
    'שעת דיון': 'hearing_time',
    'אולם דיון': 'hearing_room',
    'סוג גורם שיפוטי בדיון': 'type_of_judicial_entity_in_hearing',
    'סטטוס דיון': 'hearing_status',
    'תיאור בקשה': 'request_description',
    'מספר הבקשה': 'request_number',
    'תאריך הבקשה': 'request_date',
    'מגיש הבקשה': 'request_submitter',
    'נדחה מהמרשם הבקשה': 'request_rejected_from_registry',
    'שעה הבקשה': 'request_time',
    'בקשות': 'requests',
    'תאריך הישיבה': 'meeting_date',
    'מספר תיק דלמטה מלא': 'full_sub_case_number',
    'שם בית משפט דלמטה': 'sub_court_name',
    'מספר תיק דלמטה': 'sub_case_number',
    'תאריך החלטה דלמטה': 'sub_case_decision_date',
    'סוג תיק דלמטה': 'sub_case_type',
    'פירוט סוג תיק דלמטה': 'sub_case_type_details',
    'סוג גורם שיפוטי דלמטה': 'sub_case_judicial_entity_type',
    'סוג בית משפט דלמטה': 'sub_case_court_type',
    'מקום בית משפט דלמטה': 'sub_case_court_location',
    'הליך מאוחד': 'consolidated_procedure',
    'שמות ההליכים המאוחדים': 'consolidated_procedure_names',
    'צדדים נוספים': 'additional_parties',
    'מספר תיק מלא': 'full_case_number',
    'מספר השופטים': 'number_of_judges',
    'ניתנה החלטה כמבוקש הבקשה': 'decision_granted_as_requested',
    'Year': 'year'
}

In [5]:
# Rename columns
df_translated = df.rename(columns=col_translation)
display(df_translated.head(10))

,case_number,number_of_decisions_in_case,procedure_name,year_of_procedure_opening,procedure_number,procedure_type,procedure_type_details,confidential_case,date_of_last_event,case_duration_in_days,...,sub_case_judicial_entity_type,sub_case_court_type,sub_case_court_location,consolidated_procedure,consolidated_procedure_names,additional_parties,full_case_number,number_of_judges,decision_granted_as_requested,year
0,7618/18,10,"ע""א 7618/18",2018,"ע""א 7618/18","ע""א",ערעור אזרחי,0.0,2019-11-12,381,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
1,6857/18,4,"ע""א 6857/18",2018,"ע""א 6857/18","ע""א",ערעור אזרחי,0.0,2021-03-18,897,...,['שופט'],['מחוזי '],['תל אביב'],1.0,"[['ע""א 6701/18', 'ע""א 6857/18']]",NaN,NaN,NaN,NaN,2018
2,5821/18,2,"ע""פ 5821/18",2018,"ע""פ 5821/18","ע""פ",ערעור פלילי,0.0,2018-11-22,108,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2018
3,2075/18,5,"ע""א 2075/18",2018,"ע""א 2075/18","ע""א",ערעור אזרחי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""א 2033/18', 'ע""א 1751/18', 'ע""א 2075/18']]",NaN,NaN,NaN,NaN,2018
4,4217/18,10,"ע""פ 4217/18",2018,"ע""פ 4217/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
5,2212/18,7,"ע""א 2212/18",2018,"ע""א 2212/18","ע""א",ערעור אזרחי,0.0,2018-12-31,243,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
6,4344/18,2,"ע""פ 4344/18",2018,"ע""פ 4344/18","ע""פ",ערעור פלילי,0.0,2019-08-18,439,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
7,1993/18,4,"ע""פ 1993/18",2018,"ע""פ 1993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""פ 1993/18', 'ע""פ 2017/18']]",NaN,NaN,NaN,NaN,2018
8,4993/18,4,"ע""פ 4993/18",2018,"ע""פ 4993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
9,3128/18,4,"ע""פ 3128/18",2018,"ע""פ 3128/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018


In [6]:
# display shape
df_shape(df_translated)

# display info
display(df_translated.info)

# number of NAs
print(f"Number of rows with empty documnet body: {df_translated['document_body'].isna().sum()}")

Data shape: 46109 rows x 140 columns


<bound method DataFrame.info of       case_number  number_of_decisions_in_case  \
0         7618/18                           10   
1         6857/18                            4   
2         5821/18                            2   
3         2075/18                            5   
4         4217/18                           10   
...           ...                          ...   
46104     7466/21                            4   
46105     8957/21                            5   
46106     3780/21                            4   
46107      234/20                            3   
46108     2829/21                            4   

                                          procedure_name  \
0                                            ע"א 7618/18   
1                                            ע"א 6857/18   
2                                            ע"פ 5821/18   
3                                            ע"א 2075/18   
4                                            ע"פ 4217/18   
...    

Number of rows with empty documnet body: 2688


# Pre-processing

Create dataframe with textual features suitable for NLP taks.

**Features**:
- Case number - for case identification
- Procedure name (case name) - includes case id and addtional info such as case type (e.g. civil/criminal)
- Date - the date of last decision or judgement
- Dcoument body - textual feature that includes the case process info, parties claims, result, etc.

**Filters**:
- Drop rows with empty document body

In [7]:
df_cleaned = df_translated[['case_number', 'procedure_name', 'date_of_last_decision_or_judgment', 'case_link', 'document_body']].dropna(subset=['document_body'])
df_shape(df_cleaned)

Data shape: 43421 rows x 5 columns


Clean the document body:
- remove start [' and end ]' enclosures
- remove trailing info in the end of the document, such as URLs and document references
- remove unicode and other undesired escape sequence chars. Example: \xa0 and \u200f (obtained after HTML parsing)
- remove multiple newlines with a space
- remove multiple back-slashes followed by \' 
- replace multiple spaces with a single space
- remove leading and trailing spaces

In [8]:
def rm_str_enclosures(i_str, start="['", end="']"):
    if i_str.startswith(start) and i_str.endswith(end):
        i_str = i_str[len(start):-len(end)]
    return i_str

def clean_document(document_text, trailing_sep='____'):
    # Remove the enclosing [' and ']
    document_text = rm_str_enclosures(document_text)
    # if document_text.startswith("['") and document_text.endswith("']"):
    #     document_text = document_text[2:-2]
    
    # Remove the unnecessary string from the end, which usually starts with multiple underscore characters
    if trailing_sep in document_text:
        document_text = document_text.split(trailing_sep)[0]
    
    # remove unicode and other undesired escape sequence chars
    for rm_str in [r'\xa0', r'\u200f']:
        document_text = document_text.replace(rm_str, '')
    # # Remove \xa0
    # document_text = document_text.replace(r'\xa0', '')

    # # remove \u200f
    # document_text = document_text.replace(r'\u200f', '')

    # Remove multiple newlines with a space
    document_text = re.sub(r'(\\n)+', ' ', document_text)
    
    # remove multiple back-slashes followed by \'
    document_text = re.sub(r"(\\)+'", "'", document_text)

    # Replace multiple spaces with a single space
    document_text = re.sub(r'\s+', ' ', document_text)

    # Remove leading and trailing spaces
    document_text = document_text.strip()
    
    return document_text

def clean_date(i_date):
    return rm_str_enclosures(i_date)


Clean the document body text

In [9]:
case_example = '5780/02'

doc_body_before = df_cleaned.loc[df_cleaned['case_number'] == case_example, 'document_body'].values[0]
df_cleaned['document_body'] = df_cleaned['document_body'].apply(clean_document)
doc_body_after = df_cleaned.loc[df_cleaned['case_number'] == case_example, 'document_body'].values[0]

print(f"Document body before:\n{doc_body_before}\nDocument body after:\n {doc_body_after}")

Document body before:
['\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\nבבית המשפט העליון בשבתו כבית משפט גבוה\nלצדק\n\n\xa0\n\nבג"ץ\xa0 5780/02\n\n\xa0\n\n\xa0\n\nבפני:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 כבוד השופט מ\' חשין\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 כבוד השופטת ד\' ביניש\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 כבוד השופט א\' ריבלין\n\n\n\n\xa0\n\nהעותר:\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0

Clean date of last decision or judgment and rename the column

In [10]:
print(f"Before:")
display(df_cleaned.head(3))

df_cleaned['date_of_last_decision_or_judgment'] = df_cleaned['date_of_last_decision_or_judgment'].apply(clean_date)
df_cleaned = df_cleaned.rename(columns={'date_of_last_decision_or_judgment': 'case_date'})

print(f"After:")
display(df_cleaned.head(3))

Before:


,case_number,procedure_name,date_of_last_decision_or_judgment,case_link,document_body
0,7618/18,"ע""א 7618/18",['2019-11-12'],https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 7618/18 בבית המשפט העליון בשב..."
1,6857/18,"ע""א 6857/18",['2021-03-18'],https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 6701/18 בבית המשפט העליון בשב..."
2,5821/18,"ע""פ 5821/18",['2018-11-21'],https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 5821/18 בבית המשפט העליון בשב..."


After:


,case_number,procedure_name,case_date,case_link,document_body
0,7618/18,"ע""א 7618/18",2019-11-12,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 7618/18 בבית המשפט העליון בשב..."
1,6857/18,"ע""א 6857/18",2021-03-18,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""א 6701/18 בבית המשפט העליון בשב..."
2,5821/18,"ע""פ 5821/18",2018-11-21,https://supremedecisions.court.gov.il/Verdicts...,"פסק דין בתיק ע""פ 5821/18 בבית המשפט העליון בשב..."


# Save Processed Data

<div class="alert alert-block alert-info">  
<b>⚠️ Info:</b> In further stpes tried to process/embedd 1K documnets using DictaLM 2.0 7B with F16 (Float16) quantization, size: ~14GB <br/>
After chunking I got ~5579 splitted text documents. Estimated embedding time is approx 209h or ~8-9 days. <br/>
Due to long computation times I reduced the number of documents to 100 (randomly picked), and moved to a smaller model DictaLM 2.0 7B with Q8(quantized 8-bit), size: ~7.7GB <br/>
Approx. run time: 100 docs = 542 chunks, embedding rate ~20sec/chunk, total time -> 542 * 20 = 10,840 sec = 180.7 min = 3h
</div>

In [11]:
# get N random cases
N, N1 = 1000, 100
df_cleaned_sampled = df_cleaned.sample(n=N, random_state=42)
df_cleaned_sub_sampled = df_cleaned_sampled.sample(n=N1, random_state=42)

processed_data = {'all': df_cleaned, '1k':  df_cleaned_sampled, '100': df_cleaned_sub_sampled}

# save data to files
for k in processed_data:
    save_file(processed_data[k], f'data/processed_data_ar_ap_{k}.xlsx')

DataFrame successfully saved to data/processed_data_ar_ap_all.xlsx
DataFrame successfully saved to data/processed_data_ar_ap_1k.xlsx
DataFrame successfully saved to data/processed_data_ar_ap_100.xlsx
